# $\beta$ -Pic b planet

I'll try to obtain the same posteriors as found in the paper of [Sun et al](https://arxiv.org/pdf/2201.08506.pdf)

## Import the necessary packages

In [7]:
from prior import prior_distributions
from simulator import OrbitCreator
from training import train_sbi

import torch

import orbitize
from orbitize import read_input

from lampe.data import H5Dataset
from lampe.inference import * # Import all methods and their losses

## Load the dataset with the observations

In [8]:
data_set = read_input.read_file('{}/betaPic.csv'.format(orbitize.DATADIR))

data_set = data_set[:-1] # Discard the  RV observation, don't know how to take it into account

## Define the priors
The priors are defined as such 
$$
\begin{align*}
a &\sim \log \mathcal{U}(10,10^{4})\\
e &\sim \mathcal{U}(10^{8}, 0.99)\\
i &\sim \mathcal{U}(0,180)\\
\omega &\sim \mathcal{U}(0,360)\\
\Omega &\sim \mathcal{U}(0,360)\\
\tau &\sim \mathcal{U}(0,1)\\
\pi &\sim \mathcal{N}(56.95, 0.26) \\
M_T &\sim \mathcal{N}(1.22,0.08)\\
\end{align*}
$$


In [9]:
prior = prior_distributions(log_uniform_lower = torch.tensor(10.0), 
                            log_uniform_upper = torch.tensor(10**4),
                            uniform_lower = torch.tensor([10e-8, 0.0, 0.0, 0.0, 0.0]), 
                            uniform_upper = torch.tensor([0.99, 180.0, 360.0, 360.0, 1.0]),
                            gaussian_mean = torch.tensor([56.95, 1.22]), 
                            gaussian_std = torch.tensor([0.26, 0.08]))

## Instantiate the simulator

And test if it works well

In [10]:
simulator = OrbitCreator(data_set)

thetas = prior.sample((1,))
x = simulator(thetas)

label_print = ['a', 'e', 'i', 'ω', 'Ω', 'τ', 'π', 'Mt']

for label, theta_value in zip(label_print, thetas[0].tolist()):
    print(f"{label}: {theta_value:.3f}")

print("\n x:")
for i in range(0, len(x[0]), 2):
    x_coord = round(x[0][i].item(), 3)
    y_coord = round(x[0][i+1].item(), 3)
    print(f"({x_coord}, {y_coord})", end="\n")

a: 39.571
e: 0.368
i: 49.861
ω: 191.044
Ω: 178.085
τ: 0.248
π: 56.776
Mt: 1.262

 x:
(1122.525, -1988.948)
(1411.975, -1989.417)
(1066.035, -2002.987)
(1054.719, -1988.102)
(1038.425, -1994.249)
(1010.405, -1989.864)
(1002.094, -1994.468)
(1001.466, -2014.087)
(989.278, -2016.897)
(979.852, -2008.134)
(1059.21, -1991.194)
(1059.057, -1997.436)
(995.189, -1996.606)
(945.078, -1995.794)
(945.098, -2000.134)
(917.872, -1992.583)
(917.881, -1998.01)
(875.938, -1977.885)
(875.414, -1998.351)
(815.719, -1994.157)
(815.713, -1990.89)
(815.371, -1991.069)
(811.567, -1991.199)
(811.574, -1990.463)
(811.401, -1990.978)
(753.742, -1988.589)
(728.394, -1983.964)
(690.113, -1983.506)
(684.982, -1979.579)
(681.976, -1979.667)
(676.665, -1979.589)
(740.296, -1986.428)
(502.691, -1957.646)
(492.195, -1949.221)


## Load the generated dataset

In [11]:
trainset = H5Dataset('Datasets/beta-pic-train.h5',
                     batch_size=2048, shuffle=True)
validset = H5Dataset('Datasets/beta-pic-val.h5',
                     batch_size=2048)
testset = H5Dataset('Datasets/beta-pic-test.h5')

## Train the SBI model

In [12]:
len_obs = len(data_set) * 2 # 2 coordinates per observation

estimator = NRE(8, len_obs, hidden_features=[256]*5).cuda()

train_sbi('NRE-test-adam', 
          estimator, 
          NRELoss, 
          trainset, 
          validset, 
          epochs = 512)

100%|██████████| 3/3 [01:21<00:00, 27.07s/epoch, loss=0.378, val_loss=0.372]
